In [1]:
import torch
import lib.utils.bookkeeping as bookkeeping
from torch.utils.data import DataLoader
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import lib.models.models as models
import lib.models.model_utils as model_utils
from lib.datasets import mnist, maze, protein, synthetic
import lib.datasets.dataset_utils as dataset_utils
import lib.losses.losses as losses
import lib.losses.losses_utils as losses_utils
import lib.training.training as training
import lib.training.training_utils as training_utils
import lib.optimizers.optimizers as optimizers
import lib.optimizers.optimizers_utils as optimizers_utils
import lib.sampling.sampling as sampling
import lib.sampling.sampling_utils as sampling_utils
import os
from lib.datasets.maze import maze_acc
from ruamel.yaml.scalarfloat import ScalarFloat
def count_turns(labyrinth):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Rechts, Unten, Links, Oben
    num_turns = 0

    for i in range(1, labyrinth.shape[0] - 1):
        for j in range(1, labyrinth.shape[1] - 1):
            if labyrinth[i, j] == 1:
                valid_neighbors = 0
                for dx, dy in directions:
                    ni, nj = i + dx, j + dy
                    if labyrinth[ni, nj] == 1:
                        valid_neighbors += 1
                if valid_neighbors == 2:  # Zwei gültige Nachbarn bedeuten eine Richtungsänderung
                    num_turns += 1

    return num_turns

In [2]:


path = 'SavedModels/MAZEunet/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2023-12-30' # 2
config_name = 'config_001_lastunet.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_399999_lastunet.pt' 



path = 'SavedModels/MAZE/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2023-12-30' # 2
config_name = 'config_001_bert.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_229999_bert.pt' 
path = 'SavedModels/MAZE/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2023-12-22' # 2
config_name = 'config_001_hollow8M.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_299999_hollow8M.pt' 

config_path = os.path.join(path, date, config_name)
checkpoint_path = os.path.join(path, date, model_name)

In [3]:
# creating models
cfg = bookkeeping.load_config(config_path)
cfg.sampler.name = 'MidPointTauL' #ExactSampling' # ElboLBJF CRMTauL CRMLBJF
cfg.sampler.num_corrector_steps = 0
cfg.sampler.corrector_entry_time = ScalarFloat(0.0)
cfg.sampler.num_steps = 10
cfg.sampler.is_ordinal = False
#print(cfg)
device = torch.device(cfg.device)

model = model_utils.create_model(cfg, device)
print("number of parameters: ", sum([p.numel() for p in model.parameters()]))

#modified_model_state = utils.remove_module_from_keys(loaded_state['model'])
#model.load_state_dict(modified_model_state)
#optimizer = optimizers_utils.get_optimizer(model.parameters(), cfg)
optimizer = torch.optim.Adam(model.parameters(), cfg.optimizer.lr)

sampler = sampling_utils.get_sampler(cfg)

state = {"model": model, "optimizer": optimizer, "n_iter": 0}
state = bookkeeping.load_state(state, checkpoint_path)
state['model'].eval()


number of parameters:  7808643
ema state dict function


In [4]:
n_samples = 1024 #cfg.data.batch_size 
samples, changes = sampler.sample(model, n_samples)
print(changes)
print(np.mean(changes))
saved_samples = samples

/home/pheller/my_python_env/lib/python3.11/site-packages/torch/nn/modules/activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(


tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
[]
nan


/home/pheller/my_python_env/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/pheller/my_python_env/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [5]:
print(np.mean(changes[1:]))

nan


In [6]:
correct_mazes = maze_acc(saved_samples)

Accuracy: From 1024 are 0.0% solvable.
Average path length: nan and prob nan%
Average wall length: nan and prob nan%
Average way length: nan and prob nan%


ValueError: need at least one array to stack

In [ ]:
cfg.data.name = 'Maze3SComplete'
cfg.data.batch_size = n_samples

if cfg.data.name == 'Maze3SComplete':
    limit = cfg.data.batch_size
    cfg.data.limit = limit 

dataset = dataset_utils.get_dataset(cfg, device)
dataloader = torch.utils.data.DataLoader(dataset,
    batch_size=cfg.data.batch_size,
    shuffle=cfg.data.shuffle)

for i in dataloader:
    true_dl = i
    c_i = maze_acc(i.cpu().numpy())
    true_dl = true_dl.reshape(cfg.data.batch_size, -1).flatten()

In [ ]:
samples = samples.reshape(cfg.data.batch_size, -1).flatten()
from scipy.stats import wasserstein_distance
print("EMD", wasserstein_distance(samples, true_dl.cpu().numpy()))
# LBJF: 0.0168, 0.024355
# MPSampler

# Hollow ELbo implicit: TauL: 60 EMD 0.01805 0.0055 LBJF: 62 0.00545 Exact: 67% 0.0098 0.00515 2.5min
# ScoreNet: LBJF 50% 0.0152 TauL: 40% 0.0098 0sek => 3 times longer 
# Unet: LBJF: 44% 0.00562 TauL: 30% 0.009856 MidPointS: 72%

# Hollow Implicit: LBJF 0.0063, 0.00266 88% TauL: 78% 0.009086 0.00335 Exact: 80% 0.00888 82% 0.000926
# LBJF 10 corrector steps: 93% 0.001: TauL: 88% 0.00148

# TauL: 81%  0.0012839506172838133

# Hollow CRM: LBJF: 10: 60% 0.006 20: 75% 0.01058 30: 82% 0.001606 50:: 84% 0.0163 + 0.00415 100:84 0.00311 250: 82% 0.0035 500: 85% 0.00400
#             TauL: 10: 20: 75% 0.021 30: 50: 80% 0.00383 100: 85% 0.0051 250: 84% 0.018 500: 84% 0.001659
# 10: 1% 20: 1% 0.0135 30: 11% 0.00567, 50: 40% 0.00567, 100: 67% 0.00438 250: 78% 0.0035111
# Analytical: 10: 55 0.0276 20: 75% 30: 78% 0.003 0.008 50: 82%  0.006 100: 82% 0.006 250: 83% 0.00105 500: 85% 0.005

# TauL: without rejecting: 10:

# MPTauL: 
# 3%, 0.123

In [ ]:
steps = np.array([10, 20, 30, 50, 100, 250, 500] )
acc_lbjf = np.array([60, 75, 80, 82, 84, 85, 88] ) / 100
acc_anal = np.array([55, 75, 78, 82, 82, 83, 85]) / 100
acc_taul = np.array([0, 1, 11, 35, 67, 78, 80]) / 100
rej_rate = np.array([0.1662, 0.0732, 0.0434, 0.0243, 0.0115, 0.003, 0.00084])
#20: 0.07 30: 50: 0.0434, 0.024, 0.0

In [ ]:
plt.plot(steps, acc_lbjf, label='Euler', marker='o') # 'o' für Kreise
plt.plot(steps, acc_anal, label='Analytical', marker='o')
plt.plot(steps, acc_taul, label='Tau-Leaping', marker='o')
plt.xlabel('NFE', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Accuracy of Sampling Methods with Varying NFE', fontsize=13)
plt.legend()
plt.savefig('accuracy_plot.png')
plt.show()

In [ ]:
plt.plot(steps, rej_rate, marker='o', color='black')
plt.ylabel('Average Rejection Rate', fontsize=12)
plt.xlabel('NFE', fontsize=12)
plt.title('Average Rejection Rate of Tau-Leaping Scheme', fontsize=13)
plt.savefig('rej_rate_plot.png')
plt.show()